In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [2]:
import xgboost
from xgboost import XGBClassifier
xgboost.__version__

'1.7.4'

In [3]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

G3_TPM_merged_lats = pd.read_csv('../../g3/G3_surface_tpm_updatedMarferret_marmicroDb2023_noOutliers_fixedTPM_fall2023.csv')

,PF00003,PF00013,PF00022,PF00034,PF00071,PF00124,PF00125,PF00139,PF00169,PF00185,...,PF14259,PF14727,PF14778,PF14782,PF14870,PF15104,PF15782,PF16158,PF16313,PF16672
0,0.000000,209.057959,376.524053,0.000000,442.980543,0.0,204.073071,0.0,163.836242,50.788932,...,0,4.147061,0.0,0.000000,0.000000,3.885303,0.0,0.000000,26.103846,0.000000
1,0.000000,679.958835,150.006542,945.754709,384.339341,0.0,105.566237,0.0,65.834551,0.000000,...,0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,8.620307,0.000000
2,0.000000,253.195582,319.001763,0.000000,483.364671,0.0,272.158731,0.0,121.346538,48.340991,...,0,1.364684,0.0,0.000000,0.000000,0.000000,0.0,0.000000,5.805452,0.000000
3,0.000000,430.489863,214.292589,873.533287,349.413348,0.0,34.101693,0.0,43.651391,0.000000,...,0,7.060303,0.0,0.000000,0.000000,6.784676,0.0,8.336889,20.354027,5.719739
4,0.000000,227.915022,148.530513,0.000000,1263.827565,0.0,62.323494,0.0,122.756264,463.996871,...,0,0.000000,0.0,0.000000,236.433417,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,0.000000,357.322018,258.251403,0.000000,398.350499,0.0,227.899056,0.0,119.799237,85.841483,...,0,0.000000,0.0,0.000000,3.613349,0.000000,0.0,0.000000,5.246771,0.000000
215,0.000000,636.062145,97.416395,762.662411,446.924284,0.0,82.858507,0.0,64.409828,0.000000,...,0,6.773310,0.0,0.000000,0.000000,0.000000,0.0,9.997506,6.508887,12.570439
216,0.000000,254.381770,212.719690,0.000000,362.134523,0.0,172.057443,0.0,118.039184,37.592403,...,0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.404482,0.000000
217,143.461827,1719.341491,128.738691,0.000000,1417.083201,0.0,56.544904,0.0,824.705613,0.000000,...,0,0.000000,0.0,50.930298,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [4]:
G3_TPM_merged_lats.columns[(G3_TPM_merged_lats == 0).all()]

Index(['PF01184', 'PF06206', 'PF07784', 'PF08713', 'PF13407', 'PF14259'], dtype='object')

In [6]:
features = features['pfam']

In [13]:
len(features)

183

In [14]:
G3_TPM_merged_lats = G3_TPM_merged_lats[features]

In [16]:
G3_TPM_merged_lats.columns[(G3_TPM_merged_lats == 0).all()]

Index(['PF14259', 'PF13407', 'PF01184', 'PF08713', 'PF06206', 'PF07784'], dtype='object')

In [17]:
train_data = train_data[features]

In [18]:
le = LabelEncoder()

In [19]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [20]:
G3 = scaler.transform(G3_TPM_merged_lats)

In [21]:
#gamma: 0.5, learning_rate: 0.2, max_depth: 3, n_estimators: 100, reg_lambda: 0.0 
model = XGBClassifier(gamma=0.5, learning_rate=0.2, max_depth=3, n_estimators=100, reg_lambda=0.0)
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [22]:
xg_predictions_G3 = model.predict(G3)

In [24]:
prob = model.predict_proba(G3)

In [25]:
G3_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_G3})

In [26]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [28]:
G3_predictions.to_csv('../../g3/G3_trophicModePredictions_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)

In [29]:
probDF.to_csv('../../g3/G3_trophicModePredictionsProbabilities_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)